In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
from binance.client import Client
import pandas as pd
import time

# ✅ Binance API Credentials (Replace with your own)
API_KEY = "MrittnLSwQXCOMEEF1By9jwKeYY5pZnXntcqfCPBUQpFv3LkPI5Zc1JXbynrvGcN"
API_SECRET = "NcSVtO5U5B3362126CEOD4JlSjRMFCDFwPOuUUgetSt40mUW1mJoDXJDylcSryk9"

# ✅ Connect to Binance API
client = Client(API_KEY, API_SECRET)

# ✅ Trading Parameters
TIMEFRAME = "15m"  # 15-minute candles
LEVERAGE = 10  # Leverage for futures trading
STOP_LOSS_PERCENT = 0.02  # 2% Stop Loss
TAKE_PROFIT_LEVELS = [1.02, 1.04, 1.06]  # TP1: +2%, TP2: +4%, TP3: +6%
TRADE_AMOUNT = 0.001  # Amount of each crypto to trade

# ✅ Function to Get USDT Pairs with High Volume
def get_usdt_pairs():
    tickers = client.get_ticker()
    usdt_pairs = [ticker['symbol'] for ticker in tickers if ticker['symbol'].endswith("USDT") and float(ticker['quoteVolume']) > 50000000]
    return usdt_pairs[:5]  # Take top 5 pairs by volume

# ✅ Set Leverage (Required for Binance Futures)
def set_leverage(symbol, leverage):
    try:
        client.futures_change_leverage(symbol=symbol, leverage=leverage)
        print(f"🔹 Leverage set to {leverage}x for {symbol}")
    except Exception as e:
        print(f"⚠️ Error setting leverage: {e}")

# ✅ Get Historical Data
def get_candles(symbol, interval, limit=50):
    klines = client.get_klines(symbol=symbol, interval=interval, limit=limit)
    df = pd.DataFrame(klines, columns=[
        'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 
        'CloseTime', 'QuoteAssetVolume', 'Trades', 'TakerBuyBase', 'TakerBuyQuote', 'Ignore'
    ])
    df = df[['Time', 'Open', 'High', 'Low', 'Close', 'Volume']].astype(float)
    return df

# ✅ Signal Generator (Identifies Entry & Exit Points)
def signal_generator(df):
    open_price = df.Open.iloc[-1]
    close_price = df.Close.iloc[-1]
    prev_open = df.Open.iloc[-2]
    prev_close = df.Close.iloc[-2]

    # 📌 Bearish Pattern (Exit Signal)
    if (open_price > close_price and prev_open < prev_close and close_price < prev_open and open_price >= prev_close):
        return "SELL"

    # 📌 Bullish Pattern (Entry Signal)
    elif (open_price < close_price and prev_open > prev_close and close_price > prev_open and open_price <= prev_close):
        return "BUY"
    
    return "HOLD"

# ✅ Execute Trade (Buy with TP & SL)
def execute_trade(symbol, order_type):
    if order_type == "BUY":
        entry_price = float(client.get_symbol_ticker(symbol=symbol)['price'])
        stop_loss_price = entry_price * (1 - STOP_LOSS_PERCENT)
        tp1 = entry_price * TAKE_PROFIT_LEVELS[0]
        tp2 = entry_price * TAKE_PROFIT_LEVELS[1]
        tp3 = entry_price * TAKE_PROFIT_LEVELS[2]

        # ✅ Place Buy Order
        order = client.futures_create_order(
            symbol=symbol,
            side=Client.SIDE_BUY,
            type=Client.ORDER_TYPE_MARKET,
            quantity=TRADE_AMOUNT
        )
        print(f"\n✅ BUY Order Placed for {symbol} at {entry_price:.2f} USDT")
        print(f"📌 Stop-Loss: {stop_loss_price:.2f} USDT")
        print(f"🎯 TP1: {tp1:.2f} USDT")
        print(f"🎯 TP2: {tp2:.2f} USDT")
        print(f"🎯 TP3: {tp3:.2f} USDT")

        # ✅ Place Take Profit and Stop-Loss Orders
        sell_qty_tp1 = round(TRADE_AMOUNT * 0.25, 6)  # 25% for TP1
        sell_qty_tp2 = round(TRADE_AMOUNT * 0.25, 6)  # 25% for TP2
        sell_qty_tp3 = round(TRADE_AMOUNT * 0.50, 6)  # 50% for TP3

        client.futures_create_order(
            symbol=symbol,
            side=Client.SIDE_SELL,
            type=Client.ORDER_TYPE_LIMIT,
            quantity=sell_qty_tp1,
            price=tp1,
            timeInForce=Client.TIME_IN_FORCE_GTC
        )

        client.futures_create_order(
            symbol=symbol,
            side=Client.SIDE_SELL,
            type=Client.ORDER_TYPE_LIMIT,
            quantity=sell_qty_tp2,
            price=tp2,
            timeInForce=Client.TIME_IN_FORCE_GTC
        )

        client.futures_create_order(
            symbol=symbol,
            side=Client.SIDE_SELL,
            type=Client.ORDER_TYPE_LIMIT,
            quantity=sell_qty_tp3,
            price=tp3,
            timeInForce=Client.TIME_IN_FORCE_GTC
        )

        # ✅ Stop-Loss Order
        client.futures_create_order(
            symbol=symbol,
            side=Client.SIDE_SELL,
            type=Client.ORDER_TYPE_STOP_MARKET,
            quantity=TRADE_AMOUNT,
            stopPrice=stop_loss_price
        )

    elif order_type == "SELL":
        exit_price = float(client.get_symbol_ticker(symbol=symbol)['price'])

        # ✅ Place Sell Order
        order = client.futures_create_order(
            symbol=symbol,
            side=Client.SIDE_SELL,
            type=Client.ORDER_TYPE_MARKET,
            quantity=TRADE_AMOUNT
        )
        print(f"\n🔻 SELL Order Placed for {symbol} at {exit_price:.2f} USDT")

# ✅ Trading Bot Function
def trading_bot():
    print("\n🚀 Trading bot running...")

    usdt_pairs = get_usdt_pairs()
    
    for symbol in usdt_pairs:
        set_leverage(symbol, LEVERAGE)
        df = get_candles(symbol, TIMEFRAME)
        signal = signal_generator(df)

        if signal == "BUY":
            execute_trade(symbol, "BUY")
        elif signal == "SELL":
            execute_trade(symbol, "SELL")
        else:
            print(f"⚠️ No trading signal detected for {symbol}")

# ✅ Scheduler to Run Every 5 Minutes
scheduler = BlockingScheduler()
scheduler.add_job(trading_bot, 'interval', minutes=5)

print("🚀 Trading bot started... Press Ctrl+C to stop.")
scheduler.start()

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
from binance.client import Client
import pandas as pd
import time

# ✅ Binance API Credentials (Replace with your own)
API_KEY = "MrittnLSwQXCOMEEF1By9jwKeYY5pZnXntcqfCPBUQpFv3LkPI5Zc1JXbynrvGcN"
API_SECRET = "NcSVtO5U5B3362126CEOD4JlSjRMFCDFwPOuUUgetSt40mUW1mJoDXJDylcSryk9"

# ✅ Connect to Binance API
client = Client(API_KEY, API_SECRET)

# ✅ Trading Parameters
TIMEFRAME = "15m"  # 15-minute candles
LEVERAGE = 10  # Leverage for futures trading
STOP_LOSS_PERCENT = 0.02  # 2% Stop Loss
TAKE_PROFIT_LEVELS = [1.02, 1.04, 1.06]  # TP1: +2%, TP2: +4%, TP3: +6%
TRADE_AMOUNT = 0.001  # Amount of each crypto to trade

# ✅ Function to Get USDT Pairs with High Volume
def get_usdt_pairs():
    tickers = client.get_ticker()
    usdt_pairs = [ticker['symbol'] for ticker in tickers if ticker['symbol'].endswith("USDT") and float(ticker['quoteVolume']) > 50000000]
    return usdt_pairs[:5]  # Take top 5 pairs by volume

# ✅ Set Leverage (Required for Binance Futures)
def set_leverage(symbol, leverage):
    try:
        client.futures_change_leverage(symbol=symbol, leverage=leverage)
        print(f"🔹 Leverage set to {leverage}x for {symbol}")
    except Exception as e:
        print(f"⚠️ Error setting leverage: {e}")

# ✅ Get Historical Data
def get_candles(symbol, interval, limit=50):
    klines = client.get_klines(symbol=symbol, interval=interval, limit=limit)
    df = pd.DataFrame(klines, columns=[
        'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 
        'CloseTime', 'QuoteAssetVolume', 'Trades', 'TakerBuyBase', 'TakerBuyQuote', 'Ignore'
    ])
    df = df[['Time', 'Open', 'High', 'Low', 'Close', 'Volume']].astype(float)
    return df

# ✅ Signal Generator (Identifies Entry & Exit Points)
def signal_generator(df):
    open_price = df.Open.iloc[-1]
    close_price = df.Close.iloc[-1]
    prev_open = df.Open.iloc[-2]
    prev_close = df.Close.iloc[-2]

    # 📌 Bearish Pattern (Exit Signal)
    if (open_price > close_price and prev_open < prev_close and close_price < prev_open and open_price >= prev_close):
        return "SELL"

    # 📌 Bullish Pattern (Entry Signal)
    elif (open_price < close_price and prev_open > prev_close and close_price > prev_open and open_price <= prev_close):
        return "BUY"
    
    return "HOLD"

# ✅ Execute Trade (Buy with TP & SL)
def execute_trade(symbol, order_type):
    if order_type == "BUY":
        entry_price = float(client.get_symbol_ticker(symbol=symbol)['price'])
        stop_loss_price = entry_price * (1 - STOP_LOSS_PERCENT)
        tp1 = entry_price * TAKE_PROFIT_LEVELS[0]
        tp2 = entry_price * TAKE_PROFIT_LEVELS[1]
        tp3 = entry_price * TAKE_PROFIT_LEVELS[2]

        # ✅ Place Buy Order
        order = client.futures_create_order(
            symbol=symbol,
            side=Client.SIDE_BUY,
            type=Client.ORDER_TYPE_MARKET,
            quantity=TRADE_AMOUNT
        )
        print(f"\n✅ BUY Order Placed for {symbol} at {entry_price:.2f} USDT")
        print(f"📌 Stop-Loss: {stop_loss_price:.2f} USDT")
        print(f"🎯 TP1: {tp1:.2f} USDT")
        print(f"🎯 TP2: {tp2:.2f} USDT")
        print(f"🎯 TP3: {tp3:.2f} USDT")

        # ✅ Place Take Profit and Stop-Loss Orders
        sell_qty_tp1 = round(TRADE_AMOUNT * 0.25, 6)  # 25% for TP1
        sell_qty_tp2 = round(TRADE_AMOUNT * 0.25, 6)  # 25% for TP2
        sell_qty_tp3 = round(TRADE_AMOUNT * 0.50, 6)  # 50% for TP3

        client.futures_create_order(
            symbol=symbol,
            side=Client.SIDE_SELL,
            type=Client.ORDER_TYPE_LIMIT,
            quantity=sell_qty_tp1,
            price=tp1,
            timeInForce=Client.TIME_IN_FORCE_GTC
        )

        client.futures_create_order(
            symbol=symbol,
            side=Client.SIDE_SELL,
            type=Client.ORDER_TYPE_LIMIT,
            quantity=sell_qty_tp2,
            price=tp2,
            timeInForce=Client.TIME_IN_FORCE_GTC
        )

        client.futures_create_order(
            symbol=symbol,
            side=Client.SIDE_SELL,
            type=Client.ORDER_TYPE_LIMIT,
            quantity=sell_qty_tp3,
            price=tp3,
            timeInForce=Client.TIME_IN_FORCE_GTC
        )

        # ✅ Stop-Loss Order
        client.futures_create_order(
            symbol=symbol,
            side=Client.SIDE_SELL,
            type=Client.ORDER_TYPE_STOP_MARKET,
            quantity=TRADE_AMOUNT,
            stopPrice=stop_loss_price
        )

    elif order_type == "SELL":
        exit_price = float(client.get_symbol_ticker(symbol=symbol)['price'])

        # ✅ Place Sell Order
        order = client.futures_create_order(
            symbol=symbol,
            side=Client.SIDE_SELL,
            type=Client.ORDER_TYPE_MARKET,
            quantity=TRADE_AMOUNT
        )
        print(f"\n🔻 SELL Order Placed for {symbol} at {exit_price:.2f} USDT")

# ✅ Trading Bot Function
def trading_bot():
    print("\n🚀 Trading bot running...")

    usdt_pairs = get_usdt_pairs()
    
    for symbol in usdt_pairs:
        set_leverage(symbol, LEVERAGE)
        df = get_candles(symbol, TIMEFRAME)
        signal = signal_generator(df)

        if signal == "BUY":
            execute_trade(symbol, "BUY")
        elif signal == "SELL":
            execute_trade(symbol, "SELL")
        else:
            print(f"⚠️ No trading signal detected for {symbol}")

# ✅ Scheduler to Run Every 5 Minutes
scheduler = BlockingScheduler()
scheduler.add_job(trading_bot, 'interval', minutes=5)

print("🚀 Trading bot started... Press Ctrl+C to stop.")
scheduler.start()